In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

In [2]:
def load_image(path):
    """ Helper function for loading in our image path"""
    
    bits = tf.io.read_file(path)
    img = tf.io.decode_jpeg(bits, channels=3)
    img = tf.cast(img, dtype=tf.float32)/255.
    return img


def draw_boxes(img, boxes, class_names, scores, threshold=0.3, use_cv=True):
"""Function for drawing our bounding boxes once given the image and faster rcnn's outputs"""

    # color for each class
    colors = np.random.uniform(0,1, size=(len(class_names), 3))
    img = img.numpy()
    if use_cv: # if using opencv to show the picture it needs to be converted to BGR channel format
        img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    
    (h,w) = img.shape[:2]
    for i in range(boxes.shape[0]):
        color = colors[i]
        if scores[i] >= threshold: # Only draw boxes where the confidence is higher than our threshold
            box = boxes[i] * np.array([h,w,h,w])
            (yi,xi,yj,xj) = box.astype("int") # Remember faster rcnn's output for bounding box coorindates are 
                                              # (ymin, xmin, ymax, xmax) not yolo's (xmin, ymin, xmax, ymax)
            
            # Drawing the rectangle and text on image
            cv.rectangle(img, (xi,yi), (xj,yj), color,2)
            y_text = yi-15 if yi-15>15 else yi+15
            label = f"{str(class_names[i])[1:]}: {(scores[i]*100):.2f}"
            cv.putText(img, label, (xi,y_text), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return img

In [4]:
class f_rcnn():
    def detect_image(path):
        """Function meant to detect a single image at a time."""
        
        img = load_image(path)
        converted_image = img[tf.newaxis, ...]
        result = detector(converted_image)
        result = {key:value.numpy() for key,value in result.items()} 
        redrawn_img = draw_boxes(img, result["detection_boxes"], result["detection_class_entities"],
                   result["detection_scores"], threshold=0.3,use_cv=True)
        cv.imshow("Image",redrawn_img)
        cv.waitKey(0)
        cv.destroyAllWindows()
    
    def detect_video(path):
        """As the name implies this function is meant to be run on video."""
        
        cap = cv.VideoCapture(path)
        while True:
            _,img = cap.read()
            img = tf.convert_to_tensor(img, dtype=tf.float32)
            converted_image = img[tf.newaxis, ...]
            result = detector(converted_image)
            result = {key:value.numpy() for key,value in result.items()}
            img = draw_boxes(img, result["detection_boxes"], result["detection_class_entities"],
                       result["detection_scores"], threshold=0.3, use_cv=True)
            cv.imshow("Vid", img)
            if cv.waitKey(1) & 0xFF == ord("q"):
                break
        
        cap.release()
        cv.destroyAllWindows()

In [5]:
# Download model from tf hub
model_link = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(model_link).signatures["default"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [8]:
f_rcnn.detect_image("images/busyStreet2.jpg")

In [ ]:
f_rcnn.detect_video("/home/walt/Videos/CarFootage.mp4")